# This Notebook prepares the MySQL Databases for the MOVES runs 

## User-defined parameters

In [29]:
pollutants = []
years = []
counties = []

year = 2050
countyid = 49049
monthid = 1
dayid = 5

## Load Libraries

In [30]:
import pandas as pd
import pathlib
import os

## Input files

In [31]:
#Referece file directory path
file_dir_path = pathlib.Path(os.getcwd(),"Input/AgePopCheckerFiles")

#Reference File Names
agedistribution_name = 'sourcetypeagedistribution_4.csv'
sourcetypeyear_name = 'sourcetypepopulation_2022_4.csv'


# Age Distribution Fixer

### This part is supposed to replace the age distributions for all vehicles with the dmv age distributions, except for vehicles 53 and 62, which should use the default age distributions

In [32]:
# Read in age distribution file
agedistribution_filepath = pathlib.Path(file_dir_path,agedistribution_name)

agedistribution_df = pd.read_csv(agedistribution_filepath)

display(agedistribution_df)


,sourceTypeID,yearID,ageID,ageFraction
0,11,1990,0,0.051387
1,11,1990,1,0.050995
2,11,1990,2,0.063651
3,11,1990,3,0.079356
4,11,1990,4,0.073790
...,...,...,...,...
25384,62,2060,26,0.018751
25385,62,2060,27,0.017305
25386,62,2060,28,0.015933
25387,62,2060,29,0.014621


In [33]:
# Filter to only records in 2050 not type 53 and 62
filtered_agetype = agedistribution_df.loc[(agedistribution_df['yearID'] == year) & (agedistribution_df['sourceTypeID'] != 53) & (agedistribution_df['sourceTypeID'] != 62)]

display(filtered_agetype)

,sourceTypeID,yearID,ageID,ageFraction
1612,11,2050,0,0.055289
1613,11,2050,1,0.055176
1614,11,2050,2,0.054251
1615,11,2050,3,0.051533
1616,11,2050,4,0.048877
...,...,...,...,...
23121,61,2050,26,0.017778
23122,61,2050,27,0.016575
23123,61,2050,28,0.015143
23124,61,2050,29,0.013856


## For some reason, Rick's script seems to load the age distributions for both vehicle types from the same file? Come back to this

# Population Checker

### This script will create a new population for 53 and 62, based on the following inputs: TDM VMT and age distributions for all source types, Moves4 default VMT and populations.  If new population is < the existing population the database will not be deleted and the original population will be in the database

In [35]:
# Read in age distribution file
sourcetypeyear_filepath = pathlib.Path(file_dir_path,sourcetypeyear_name)

sourcetypeyear_df = pd.read_csv(sourcetypeyear_filepath)

display(sourcetypeyear_df)


,yearID,countyid,sourceTypeID,pop
0,2014,49001,11,109.000
1,2015,49001,11,104.000
2,2016,49001,11,102.000
3,2017,49001,11,124.000
4,2018,49001,11,136.000
...,...,...,...,...
17714,2056,49057,62,974.196
17715,2057,49057,62,978.659
17716,2058,49057,62,983.141
17717,2059,49057,62,987.644


In [37]:
# Filter to only records in 2050 not type 53 and 62
filtered_sourcetype = sourcetypeyear_df.loc[(sourcetypeyear_df['yearID'] == year) & (sourcetypeyear_df['countyid'] == countyid)]

display(filtered_sourcetype)

,yearID,countyid,sourceTypeID,pop
14700,2050,49049,11,17227.400
14747,2050,49049,21,103123.000
14794,2050,49049,31,388007.000
14841,2050,49049,32,44533.100
14888,2050,49049,41,750.023
14935,2050,49049,42,105.885
14982,2050,49049,43,927.970
15029,2050,49049,51,167.327
15076,2050,49049,52,29472.900
15123,2050,49049,53,4748.740


In [77]:
#VMT Calc 1 sums the sourcetype df by groups: 11, 21-32, 41-43, 51-54, and 61-62
group_dict = {
    11: [11],
    21: [21,31,32],
    31: [21,31,32],
    32: [21,31,32],
    41: [41,42,43],
    42: [41,42,43],
    43: [41,42,43],
    51: [51,52,53,54],
    52: [51,52,53,54],
    53: [51,52,53,54],
    54: [51,52,53,54],
    61: [61,62],
    62: [61,62]

}

vmt_calc_1 = filtered_sourcetype.copy()

for sourcetype in filtered_sourcetype['sourceTypeID']:
    filtered_df = filtered_sourcetype[filtered_sourcetype['sourceTypeID'].isin(group_dict[sourcetype])]
    group_pop = filtered_df['pop'].sum()
    vmt_calc_1.loc[vmt_calc_1['sourceTypeID'] == sourcetype,'pop'] = group_pop

display(vmt_calc_1)





,yearID,countyid,sourceTypeID,pop
14700,2050,49049,11,17227.400
14747,2050,49049,21,535663.100
14794,2050,49049,31,535663.100
14841,2050,49049,32,535663.100
14888,2050,49049,41,1783.878
14935,2050,49049,42,1783.878
14982,2050,49049,43,1783.878
15029,2050,49049,51,37620.057
15076,2050,49049,52,37620.057
15123,2050,49049,53,37620.057


In [ ]:
#Create VMT_calc_02

#First, create htpms vtypeday

